In [1]:
from langchain.memory.chat_message_histories import ZepChatMessageHistory
from langchain.retrievers import ZepRetriever
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.schema import HumanMessage, AIMessage
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool
from zep_python import Memory, Message, MemorySearchPayload
from uuid import uuid4
import pprint

In [2]:
# Set this to your Zep server URL
ZEP_API_URL = "http://localhost:8000"

session_id = str(uuid4())  # This is a unique identifier for the user

In [3]:
# Load your OpenAI key from a .env file
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
search = WikipediaAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to search online for answers. You should ask targeted questions",
    ),
]

# Set up Zep Chat History
zep_chat_history = ZepChatMessageHistory(
    session_id=session_id,
    url=ZEP_API_URL,
)

# Use a standard ConversationBufferMemory to encapsulate the Zep chat history
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=zep_chat_history
)

# Initialize the agent
llm = OpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

In [5]:
# Preload some messages into the memory. The default message window is 12 messages. We want to push beyond this to demonstrate auto-summarization.
test_history = [
    {"role": "human", "content": "Who was Octavia Butler?"},
    {
        "role": "ai",
        "content": (
            "Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was an American"
            " science fiction author."
        ),
    },
    {"role": "human", "content": "Which books of hers were made into movies?"},
    {
        "role": "ai",
        "content": (
            "The most well-known adaptation of Octavia Butler's work is the FX series"
            " Kindred, based on her novel of the same name."
        ),
    },
    {"role": "human", "content": "Who were her contemporaries?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler's contemporaries included Ursula K. Le Guin, Samuel R."
            " Delany, and Joanna Russ."
        ),
    },
    {"role": "human", "content": "What awards did she win?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur"
            " Fellowship."
        ),
    },
    {
        "role": "human",
        "content": "Which other women sci-fi writers might I want to read?",
    },
    {
        "role": "ai",
        "content": "You might want to read Ursula K. Le Guin or Joanna Russ.",
    },
    {
        "role": "human",
        "content": (
            "Write a short synopsis of Butler's book, Parable of the Sower. What is it"
            " about?"
        ),
    },
    {
        "role": "ai",
        "content": (
            "Parable of the Sower is a science fiction novel by Octavia Butler,"
            " published in 1993. It follows the story of Lauren Olamina, a young woman"
            " living in a dystopian future where society has collapsed due to"
            " environmental disasters, poverty, and violence."
        ),
    },
]

for msg in test_history:
    zep_chat_history.add_message(
        HumanMessage(content=msg["content"])
        if msg["role"] == "human"
        else AIMessage(content=msg["content"])
    )

# Inspect the Zep memory
**Note the summary, and that the history has been enriched with entities, token counts, UUIDs, and timestamps.**

Summaries are biased towards the most recent messages.

In [6]:
def print_messages(messages):
    for m in messages:
        print(m.dict())


print(zep_chat_history.zep_summary)
print("\n")
print_messages(zep_chat_history.zep_messages)

The human asks about Octavia Butler and the AI explains she was a science fiction author who passed away in 2006. The human then inquires which of her books were adapted into movies, and the AI mentions the FX series Kindred. Finally, the human asks about Butler's contemporaries and the AI names Ursula K. Le Guin, Samuel R. Delany, and Joanna Russ.


{'role': 'human', 'content': 'What awards did she win?', 'uuid': '7738fecd-169e-41bf-ab97-d175f5e329d7', 'created_at': '2023-06-03T22:00:05.961342Z', 'token_count': 8, 'metadata': {'system': {'entities': []}}}
{'role': 'ai', 'content': 'Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur Fellowship.', 'uuid': 'd3ce9900-0b70-4fea-b269-0cec1de55212', 'created_at': '2023-06-03T22:00:05.97513Z', 'token_count': 21, 'metadata': {'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 14, 'Start': 0, 'Text': 'Octavia Butler'}], 'Name': 'Octavia Butler'}, {'Label': 'WORK_OF_ART', 'Matches': [{'End': 33, 'Start': 19, 'Tex

In [7]:
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(zep_chat_history.zep_messages[1].dict())

{   'content': 'Octavia Butler won the Hugo Award, the Nebula Award, and the '
               'MacArthur Fellowship.',
    'created_at': '2023-06-03T22:00:05.97513Z',
    'metadata': {   'system': {   'entities': [   {   'Label': 'PERSON',
                                                      'Matches': [   {   'End': 14,
                                                                         'Start': 0,
                                                                         'Text': 'Octavia '
                                                                                 'Butler'}],
                                                      'Name': 'Octavia Butler'},
                                                  {   'Label': 'WORK_OF_ART',
                                                      'Matches': [   {   'End': 33,
                                                                         'Start': 19,
                                                                         'Tex

# Use Langchain Retriever to get relevant documents
### Specifying a hybrid query that uses both a semantic text query and matches on metadata

Typically a retriever is used in a chain. The chain would need to be customized to take advantage of hybrid search.

In [8]:
retriever = ZepRetriever(url=ZEP_API_URL, session_id=session_id, top_k=5)

retriever.get_relevant_documents(
    "Hugo Award",
    metadata={
        "where": {"jsonpath": '$.system.entities[*] ? (@.Label == "WORK_OF_ART")'}
    },
)

[Document(page_content='Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur Fellowship.', metadata={'score': 0.8514928881473501, 'uuid': 'd3ce9900-0b70-4fea-b269-0cec1de55212', 'created_at': '2023-06-03T22:00:05.97513Z', 'role': 'ai', 'metadata': {'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 14, 'Start': 0, 'Text': 'Octavia Butler'}], 'Name': 'Octavia Butler'}, {'Label': 'WORK_OF_ART', 'Matches': [{'End': 33, 'Start': 19, 'Text': 'the Hugo Award'}], 'Name': 'the Hugo Award'}, {'Label': 'EVENT', 'Matches': [{'End': 81, 'Start': 57, 'Text': 'the MacArthur Fellowship'}], 'Name': 'the MacArthur Fellowship'}]}}, 'token_count': 21}),
 Document(page_content="Write a short synopsis of Butler's book, Parable of the Sower. What is it about?", metadata={'score': 0.7407543628943766, 'uuid': '32d6ecfc-3afe-448a-a50b-0a366f501c9f', 'created_at': '2023-06-03T22:00:05.996972Z', 'role': 'human', 'metadata': {'system': {'entities': [{'Label': 'ORG', 'Matches': [{'En

# Run the agent

In [9]:
agent_chain.run(
    input="Recommend science fiction books I should read that are similar to Octavia Butler's Parable of the Sower. Research online."
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Search
Action Input: Science fiction books similar to Octavia Butler's Parable of the Sower
Observation: Page: Octavia E. Butler
Summary: Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was an American science fiction author and a multiple recipient of the Hugo and Nebula awards. In 1995, Butler became the first science-fiction writer to receive a MacArthur Fellowship.Born in Pasadena, California, Butler was raised by her widowed mother. Extremely shy as a child, Butler found an outlet at the library reading fantasy, and in writing. She began writing science fiction as a teenager. She attended community college during the Black Power movement. While  participating in a local writer's workshop, she was encouraged to attend the Clarion Workshop, then held in Pennsylvania, which focused on science fiction.She soon sold her first stories and by the late 1970s had become sufficiently successful

"Based on Octavia Butler's Parable of the Sower, you might want to read Ursula K. Le Guin's The Dispossessed, Samuel R. Delany's Dhalgren, and Joanna Russ's The Female Man."

# Calling the Zep Search API directly from the ZepChatMessageHistory object

In [10]:
zep_chat_history.search(
    "Octavia Butler",
    metadata={
        "where": {"jsonpath": '$.system.entities[*] ? (@.Label == "WORK_OF_ART")'}
    },
)

[MemorySearchResult(message={'uuid': 'd3ce9900-0b70-4fea-b269-0cec1de55212', 'created_at': '2023-06-03T22:00:05.97513Z', 'role': 'ai', 'content': 'Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur Fellowship.', 'metadata': {'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 14, 'Start': 0, 'Text': 'Octavia Butler'}], 'Name': 'Octavia Butler'}, {'Label': 'WORK_OF_ART', 'Matches': [{'End': 33, 'Start': 19, 'Text': 'the Hugo Award'}], 'Name': 'the Hugo Award'}, {'Label': 'EVENT', 'Matches': [{'End': 81, 'Start': 57, 'Text': 'the MacArthur Fellowship'}], 'Name': 'the MacArthur Fellowship'}]}}, 'token_count': 21}, metadata=None, summary=None, dist=0.911757091625383),
 MemorySearchResult(message={'uuid': '4b28dffb-a072-4a54-80b8-d718ce263aa1', 'created_at': '2023-06-03T22:00:30.122681Z', 'role': 'ai', 'content': "Based on Octavia Butler's Parable of the Sower, you might want to read Ursula K. Le Guin's The Dispossessed, Samuel R. Delany's Dhalgren, and Joann

# Filtering by Message Creation Date

In [11]:
zep_chat_history.search(
    "Octavia Butler",
    metadata={
        "start_date": "2023-06-02",
        "end_date": "2023-06-04",
        "where": {"jsonpath": '$.system.entities[*] ? (@.Label == "WORK_OF_ART")'},
    },
)

[MemorySearchResult(message={'uuid': 'd3ce9900-0b70-4fea-b269-0cec1de55212', 'created_at': '2023-06-03T22:00:05.97513Z', 'role': 'ai', 'content': 'Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur Fellowship.', 'metadata': {'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 14, 'Start': 0, 'Text': 'Octavia Butler'}], 'Name': 'Octavia Butler'}, {'Label': 'WORK_OF_ART', 'Matches': [{'End': 33, 'Start': 19, 'Text': 'the Hugo Award'}], 'Name': 'the Hugo Award'}, {'Label': 'EVENT', 'Matches': [{'End': 81, 'Start': 57, 'Text': 'the MacArthur Fellowship'}], 'Name': 'the MacArthur Fellowship'}]}}, 'token_count': 21}, metadata=None, summary=None, dist=0.9117159078908468),
 MemorySearchResult(message={'uuid': '4b28dffb-a072-4a54-80b8-d718ce263aa1', 'created_at': '2023-06-03T22:00:30.122681Z', 'role': 'ai', 'content': "Based on Octavia Butler's Parable of the Sower, you might want to read Ursula K. Le Guin's The Dispossessed, Samuel R. Delany's Dhalgren, and Joan

# Add and search for your own Custom Metadata
You can add your own custom metadata to the Zep memory. This metadata can be used to filter search results, either calling the Zep Search API directly or using the ZepRetriever.

In [12]:
zep_client = zep_chat_history.zep_client

zep_client.add_memory(
    session_id=session_id,
    memory_messages=Memory(
        messages=[Message(role="human", content="I've read many books written by Octavia Butler.", metadata={"foo": "bar"})]
    ),
)

'OK'

In [13]:
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(zep_chat_history.zep_messages[-1].dict())

{   'content': "I've read many books written by Octavia Butler.",
    'created_at': '2023-06-03T22:00:43.034056Z',
    'metadata': {   'foo': 'bar',
                    'system': {   'entities': [   {   'Label': 'PERSON',
                                                      'Matches': [   {   'End': 46,
                                                                         'Start': 32,
                                                                         'Text': 'Octavia '
                                                                                 'Butler'}],
                                                      'Name': 'Octavia '
                                                              'Butler'}]}},
    'role': 'human',
    'token_count': 13,
    'uuid': '8f3a06dd-0625-41da-a2af-b549f2056b3f'}


In [14]:
r = zep_client.search_memory(
    session_id=session_id,
    search_payload=MemorySearchPayload(
        query="Famous sci-fi authors",
        metadata={
            "where": {"jsonpath": '$[*] ? (@.foo == "bar")'},
        },
    ),
)

pp.pprint(r[0].dict())

{   'dist': 0.7170433826192629,
    'message': {   'content': "I've read many books written by Octavia Butler.",
                   'created_at': '2023-06-03T22:00:43.034056Z',
                   'metadata': {   'foo': 'bar',
                                   'system': {   'entities': [   {   'Label': 'PERSON',
                                                                     'Matches': [   {   'End': 46,
                                                                                        'Start': 32,
                                                                                        'Text': 'Octavia '
                                                                                                'Butler'}],
                                                                     'Name': 'Octavia '
                                                                             'Butler'}]}},
                   'role': 'human',
                   'token_count': 13,
                  

## Or with a Langchain retriever called from a custom chain

In [15]:
retriever.get_relevant_documents(
    "Famous sci-fi authors", metadata={"where": {"jsonpath": '$[*] ? (@.foo == "bar")'}}
)

[Document(page_content="I've read many books written by Octavia Butler.", metadata={'score': 0.8346713396773939, 'uuid': '8f3a06dd-0625-41da-a2af-b549f2056b3f', 'created_at': '2023-06-03T22:00:43.034056Z', 'role': 'human', 'metadata': {'foo': 'bar', 'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 46, 'Start': 32, 'Text': 'Octavia Butler'}], 'Name': 'Octavia Butler'}]}}, 'token_count': 13})]